In [ ]:
!pip install matplotlib pandas seaborn jupyter openpyxl

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get dataset path
dataset_path = os.getenv("DATASET_PATH")

In [ ]:
df = pd.read_excel(dataset_path)

# Display the first few rows
df.head()

In [ ]:
# Check the DataFrame info
df.info()

# Check for missing values
df.isnull().sum()


In [12]:
# Convert 'Purchase Date' to datetime
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])

# Drop duplicates
df.drop_duplicates(inplace=True)

# Handle missing values (if any)
# For example, drop rows with missing values
df.dropna(inplace=True)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Aggregate sales over time
sales_over_time = df.groupby('Purchase Date')['Sales'].sum().reset_index()

# Set the figure size and style
plt.figure(figsize=(14, 7), dpi=100)
sns.set_style('whitegrid')
sns.set_palette('colorblind')

# Plot the sales over time
plt.plot(sales_over_time['Purchase Date'], sales_over_time['Sales'], linewidth=2, color='#1f77b4')

# Format the x-axis for dates
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.xticks(rotation=45)

# Add labels and title
plt.title('Sales Over Time', fontsize=22, fontweight='bold')
plt.xlabel('Purchase Date', fontsize=18)
plt.ylabel('Total Sales', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Add gridlines
plt.grid(visible=True, which='major', linestyle='--', linewidth=0.5)

# Remove top and right spines
sns.despine()

# Show the plot
plt.savefig('sales_over_time.png', dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()


In [ ]:
# Calculate sales by category
category_sales = df.groupby('Category')['Sales'].sum().reset_index()

# Set figure size
plt.figure(figsize=(10, 6), dpi=100)

# Create barplot
sns.barplot(x='Category', y='Sales', data=category_sales, palette='pastel')

# Add labels and title
plt.title('Sales by Category', fontsize=22, fontweight='bold')
plt.xlabel('Category', fontsize=18)
plt.ylabel('Total Sales', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Annotate bars
for index, row in category_sales.iterrows():
    plt.text(index, row['Sales'] + 5000, f"${row['Sales']:,.0f}", ha='center', fontsize=14)

# Remove spines
sns.despine()

# Show plot
plt.savefig('sales_by_category.png', dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()


In [ ]:
# Set figure size
plt.figure(figsize=(12, 8), dpi=100)

# Create scatter plot
sns.scatterplot(x='Discount', y='Profit', data=df, hue='Category', alpha=0.7, edgecolor='w', s=100)

# Add regression line
sns.regplot(x='Discount', y='Profit', data=df, scatter=False, color='black', line_kws={'linewidth':2})

# Add labels and title
plt.title('Profit vs. Discount', fontsize=22, fontweight='bold')
plt.xlabel('Discount', fontsize=18)
plt.ylabel('Profit', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Add legend
plt.legend(title='Category', fontsize=12, title_fontsize=14)

# Remove spines
sns.despine()

# Show plot
plt.savefig('profit_discount_ratio.png', dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()


In [ ]:
# Calculate correlation
corr = df[['Sales', 'Quantity', 'Discount', 'Profit']].corr()

# Set figure size
plt.figure(figsize=(8, 6), dpi=100)

# Create heatmap
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', annot_kws={"size":14}, linewidths=0.5)

# Add title
plt.title('Correlation Matrix', fontsize=22, fontweight='bold')

# Adjust ticks
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Show plot
plt.savefig('correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()


In [ ]:
# Calculate sales by region
region_sales = df.groupby('Region')['Sales'].sum().reset_index().sort_values('Sales', ascending=True)

# Set figure size
plt.figure(figsize=(12, 8), dpi=100)

# Create horizontal barplot
sns.barplot(x='Sales', y='Region', data=region_sales, palette='magma')

# Add labels and title
plt.title('Sales by Region', fontsize=22, fontweight='bold')
plt.xlabel('Total Sales', fontsize=18)
plt.ylabel('Region', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Annotate bars
for index, row in region_sales.iterrows():
    plt.text(row['Sales'] + 5000, index, f"${row['Sales']:,.0f}", va='center', fontsize=14)

# Remove spines
sns.despine(left=True, bottom=True)

# Show plot
plt.savefig('sales_by_region.png', dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()


In [ ]:
# Calculate profit margin
df['Profit Margin'] = (df['Profit'] / df['Sales']) * 100

# Average profit margin by category
margin_by_category = df.groupby('Category')['Profit Margin'].mean().reset_index()

# Plot
plt.figure(figsize=(10, 6), dpi=100)
sns.barplot(x='Category', y='Profit Margin', data=margin_by_category, palette='Set2')

# Formatting
plt.title('Average Profit Margin by Category', fontsize=22, fontweight='bold')
plt.xlabel('Category', fontsize=18)
plt.ylabel('Profit Margin (%)', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Annotate bars
for index, row in margin_by_category.iterrows():
    plt.text(index, row['Profit Margin'] + 0.5, f"{row['Profit Margin']:.1f}%", ha='center', fontsize=14)

sns.despine()
plt.savefig('avg_profit_by_category.png', dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()


In [ ]:
# Calculate monthly sales
monthly_sales = df.set_index('Purchase Date').resample('M')['Sales'].sum()

# Calculate growth rate
growth_rate = monthly_sales.pct_change().fillna(0) * 100

# Plot
plt.figure(figsize=(14, 7), dpi=100)
sns.lineplot(x=monthly_sales.index, y=growth_rate, marker='o', linewidth=2)

# Formatting
plt.title('Monthly Sales Growth Rate', fontsize=22, fontweight='bold')
plt.xlabel('Month', fontsize=18)
plt.ylabel('Growth Rate (%)', fontsize=18)
plt.xticks(rotation=45, fontsize=14)
plt.yticks(fontsize=14)
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.grid(True)
sns.despine()
plt.savefig('monthly_sales_growth_rate.png', dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()


In [ ]:
#done